In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

## Импорт БД

In [2]:
df = pd.read_csv(fr"C:\ML\DataBases\russian_toxic_comments.csv", sep=",")
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


## Разделение данных 

In [3]:
test_df = df.sample(1000, random_state=0)
test_df

,comment,toxic
9470,"Светофоры работают на автоблокировке, там где ...",0.0
3683,Общая знакомая просто не попала в эту половину\n,0.0
4168,Откуда такой акцент на Москве? У нас между про...,0.0
9524,Спасибо за ответ! Очень подробно и доходчиво.\n,0.0
7994,"Ну вопрос то в том и состоит, как долго эта бе...",0.0
...,...,...
2476,Хохлушка же даже бесплатно Возьмет в рот твой ...,1.0
4070,"Как установил суд, банк занизил полную стоимос...",0.0
13142,Ставить что то на комплектные саморезы ? Серье...,0.0
7201,Ну если святую воду взбить с елеем то таки СОЖ...,0.0


In [4]:
df = df.drop(test_df.index)
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14406,Потому что запад прошел эту хуйню еще пару сот...,0.0
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0


In [5]:
val_df = df.sample(500, random_state=0)
val_df

,comment,toxic
12569,Лучший камент в этой ветке!\n,1.0
11069,"За крыжопник и двор, аардом в упор.\n",1.0
4554,Так закат может длиться не одну сотню лет. Но ...,0.0
6646,"Это вообще нонсенс, что мы все со спокойной ду...",0.0
14383,Что. ты. несёшь. дегенерат?\n,1.0
...,...,...
13326,"Люди путешествующие и или пожившие еще где то,...",0.0
1360,И первый магазин из списка который следует пос...,0.0
12119,"Вот мне интересно, я ещё в 18 мимо армии проле...",0.0
6037,"больного синдромом патау Лскал ролик, в которо...",1.0


In [6]:
train_df = df.drop(val_df.index)
train_df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14406,Потому что запад прошел эту хуйню еще пару сот...,0.0
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0


## Обработка текста

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['comment'].to_list())

In [8]:
train_sequences = tokenizer.texts_to_sequences(train_df['comment'].to_list())
test_sequences = tokenizer.texts_to_sequences(test_df['comment'].to_list())
val_sequences = tokenizer.texts_to_sequences(val_df['comment'].to_list())

In [9]:
maxlen = 300  # максимальная длина текста

padded_train_sequences = pad_sequences(train_sequences, maxlen=maxlen)
padded_test_sequences = pad_sequences(test_sequences, maxlen=maxlen)
padded_val_sequences = pad_sequences(val_sequences, maxlen=maxlen)

## Создание модели

In [10]:
output_dim = 100  # максимальное количество обрабатываемых данных

In [16]:
model = Sequential([
    layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=output_dim),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(1, activation='sigmoid')
])

In [17]:
model.compile(
    optimizer='SGD',
    loss='binary_crossentropy',
    metrics=[keras.metrics.BinaryAccuracy(),
             keras.metrics.Recall(),
             keras.metrics.Precision()]
)

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         6393100   
                                                                 
 bidirectional (Bidirection  (None, 64)                34048     
 al)                                                             
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6427213 (24.52 MB)
Trainable params: 6427213 (24.52 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0, min_delta=0.01)]

In [20]:
history = model.fit(
    padded_train_sequences,
    train_df['toxic'].to_numpy(),
    validation_data=(padded_val_sequences, val_df['toxic'].to_numpy()),
    batch_size=64,
    callbacks=callbacks,
    epochs=20
)

Epoch 1/20

 69/202 [=========>....................] - ETA: 13s - loss: 0.6700 - binary_accuracy: 0.6569 - recall_1: 0.0187 - precision_1: 0.4000

KeyboardInterrupt: 

## Проверка обучения

In [58]:
model.evaluate(padded_test_sequences, test_df['toxic'].to_numpy())

32/32 [==============================] - 0s 9ms/step - loss: 10.2627 - binary_accuracy: 0.3270 - recall_7: 1.0000 - precision_7: 0.3270


[10.262738227844238, 0.3269999921321869, 1.0, 0.3269999921321869]

## Тестирование модели

In [59]:
sms = ['Верблюдов-то за что? Дебилы, бл...',
       'Прогноз на сегодня - солнечно',
       'Я убью тебя!',
       'Я люблю тебя!',
       'Кодишь отвратительно как старый пердун!']

In [60]:
sms = tokenizer.texts_to_sequences(sms)

In [61]:
sms = pad_sequences(sms, maxlen=maxlen)

In [62]:
model.predict(sms)

1/1 [==============================] - 0s 400ms/step


array([[1.412767 ],
       [1.4083337],
       [1.4056414],
       [1.4021809],
       [1.4057677]], dtype=float32)